In [1]:
import numpy as np
import numpy.random as rnd

In [2]:
n_states = 2
n_actions = 2

In [7]:
def generate_rnd_problem(n_states, n_actions):
    P = rnd.random((n_states * n_actions, n_states))**2
    P = P/P.sum(axis=1, keepdims=True)
    r = rnd.random((n_states * n_actions, 1))
    d0 = rnd.random((n_states, 1))
    d0 /= d0.sum()
    return P, r, d0

P, r, d0 = generate_rnd_problem(n_states, n_actions)
P.shape, r.shape

((4, 2), (4, 1))

In [9]:
def generate_Mpi(n_states, n_actions, ps):
    """
    A policy is represented by a block diagonal |S| x |S||A| matrix M.

    Args:
        n_states (int): the number of states
        n-actions (int): the number of actions
        ps (array[n_states, n_actions]): the probabilities of taking action a in state s.

    Returns:
        (array[n_states, n_states x n_actions])
    """
    A = np.ones((1, n_actions))
    S = np.eye(n_states)

    M_pi = np.zeros((n_states, n_states * n_actions))
    M_pi[np.where(np.equal(1, np.kron(S, A)))] = ps.reshape(-1)
    return M_pi

def pi(M_pi, s, a):
    """
    Let state s be indexed by i and the action a be indexed by j.
    Then we have that M[i, i x |A| + j] = pi(a|s)
    """
    return M_pi[s, s*n_actions+a]

def normalise(x, axis):
    return x/np.sum(x, axis=axis, keepdims=True)

def exp_dist(x, lambda_=3.5):  # why 3.5!?
    return lambda_*np.exp(-lambda_*x)

def uniform_simplex(shape):
    # god damn. it's not as simple as I thought to generate uniform distributions on simplexs
    # https://cs.stackexchange.com/questions/3227/uniform-sampling-from-a-simplex
    # http://www.cs.cmu.edu/~nasmith/papers/smith+tromble.tr04.pdf
    return normalise(exp_dist(rnd.random(shape)),axis=1)

def generate_rnd_policy(n_states, n_actions):
    return generate_Mpi(n_states, n_actions, uniform_simplex((n_states, n_actions)))



Want to calculate the expected state distribution under a policy / MDP.

$$
\begin{align}
Pr^{\pi}(\tau) &= d_0(s_0)\pi(a_0|s_0)P(s_1|s_0,a_0)\pi(a_1|s_1)P(s_2|s_1,a_1) \dots \tag{prob of a trajectory}\\
Pr^{\pi}(s_t = s) &= \sum_{i=0}^{\mid T\mid} Pr(T[i] | T[i]_t=s) \tag{sum over traj sharing $s_t$}\\
d^{\pi}(s) &= (1-\gamma)\sum_{t=0}^{\infty} \gamma^t Pr^{\pi}(s =s_t) \\
\end{align}
$$

For a synchronous, finite state MDP, we can write this in terms of state vectors, $s$ and policy conditioned transtition matrices, $P_{\pi}$. By using $p(s_{t+1}) = P_{\pi}p(s_t)$

$$
\begin{align}
d^{\pi}(s) &= (1-\gamma)\sum_{t=0}^{\infty} \gamma^t P_{\pi}p(s) \\
&= (1-\gamma)(P_{\pi}p(s) + yP_{\pi}P_{\pi}p(s) + yyP_{\pi}P_{\pi}P_{\pi}p(s) + \dots) \\
&= (1-\gamma)(P_{\pi} + yP_{\pi}P_{\pi} + yyP_{\pi}P_{\pi}P_{\pi} + \dots )p(s) \\
&= (1-\gamma)(I - yP_{\pi})^{-1} p(s) \\
\end{align}
$$

But this doesnt work when $\pi$ is not constant?!

$$
\begin{align}
d^{\pi}(s) &= (1-\gamma)\sum_{t=0}^{\infty} \gamma^t P_{\pi(t)}p(s) \\
&= (1-\gamma)(P_{\pi(0)}p(s) + yP_{\pi(1)}P_{\pi(0)}p(s) + yyP_{\pi(2)}P_{\pi(1)}P_{\pi(0)}p(s) + \dots) \\
&= ??
\end{align}
$$

In [14]:
def state_visitation_distribution(M_pi, discount, d_0, P):
    """
    Ps + yPPs + yyPPPs + yyyPPPPs ...
    (P + yPP + yyPPP + yyyPPPP ... )s
    (I - yP)^-1 s
    """
    n = d_0.size
    P_pi = np.dot(M_pi, P)
    
    # check we have been given normalised distributions
    assert np.isclose(d_0/d_0.sum(), d_0).all()
    assert np.isclose(P_pi/P_pi.sum(axis=1), P_pi).all()
    
    return (1-discount)*np.dot(np.linalg.inv(np.eye(n) - discount * P_pi), d_0)

In [15]:
M_pi = generate_rnd_policy(n_states, n_actions)
state_visitation_distribution(M_pi, 0.9, d0, P)

array([[0.55765921],
       [0.50996122]])

In [6]:
d_empirical = (1-0.9)*sum([np.dot((0.9**(i-1))*np.linalg.matrix_power(P, i), s) 
                   for i in range(1, 100000)])
d_empirical

LinAlgError: Last 2 dimensions of the array must be square

In [ ]:
def entropy(d):
    b = d.size
    return - np.sum(d * np.log(d) / np.log(b)) 

def max_entropy_exploration(ds_0t):
    g = dH(ds_0t, M_pi)
    return M_pi - lr * g

In [ ]:
def count_based_exploration(ds_0t, P):
    # pick a policy that chooses actions
    # with high probabilty if they have not been visited frequently
    # P(s' | s, a) . (1-d(s'))
    return np.einsum(P, (1-ds_0t), 'ijk,i->jk')

In [ ]:
def udb_exploration(ds_0t):
    # the confidence in what!?
    return 

In [19]:
d_0 = normalise(np.random.random((n_states, 1)), 0)
M_pi = generate_rnd_policy(n_states, n_actions)
P, r = generate_rnd_problem(n_states, n_actions)

d_pi = discounted_state_visitation_distribution(d_0, M_pi, P)
np.sum(d_pi)

0.09355220951751211

Neumann series

$$
(Id - T)^{-1} = \sum_{k=0}^{\infty} T^k \\
$$

In [17]:
T = 0.9 * np.dot(M_pi, P)
seq = sum([np.linalg.matrix_power(T, i) for i in range(1000)])
inv = np.linalg.inv(np.eye(n_states) - T)
np.isclose(seq, inv).all()

True

In [22]:
M = np.dot(M_pi, P)
np.linalg.matrix_power(M, 1000)

array([[0.68985908, 0.31014092],
       [0.68985908, 0.31014092]])

In [38]:
u, s, v = np.linalg.svd(M)
u[:,0]*s[0]
# u[1]*s[1]

array([-0.80563312, -0.62836267])

In [36]:
help(np.linalg.svd)

Help on function svd in module numpy.linalg.linalg:

svd(a, full_matrices=True, compute_uv=True)
    Singular Value Decomposition.
    
    When `a` is a 2D array, it is factorized as ``u @ np.diag(s) @ vh
    = (u * s) @ vh``, where `u` and `vh` are 2D unitary arrays and `s` is a 1D
    array of `a`'s singular values. When `a` is higher-dimensional, SVD is
    applied in stacked mode as explained below.
    
    Parameters
    ----------
    a : (..., M, N) array_like
        A real or complex array with ``a.ndim >= 2``.
    full_matrices : bool, optional
        If True (default), `u` and `vh` have the shapes ``(..., M, M)`` and
        ``(..., N, N)``, respectively.  Otherwise, the shapes are
        ``(..., M, K)`` and ``(..., K, N)``, respectively, where
        ``K = min(M, N)``.
    compute_uv : bool, optional
        Whether or not to compute `u` and `vh` in addition to `s`.  True
        by default.
    
    Returns
    -------
    u : { (..., M, M), (..., M, K) } array
      